In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install transformers pandas

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in c:\users\harsh\appdata\local\programs\python\python313\lib\site-packages (4.52.3)



In [5]:
import torch
print(torch.__version__)

2.7.0+cpu


In [6]:
python sentiment_analysis.py

SyntaxError: invalid syntax (4028811695.py, line 1)

In [7]:
pip install torch transformers pandas

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

# Sample dataset (replace with your own dataset, e.g., X posts or reviews)
data = {
    'text': [
        "Love the camera on this phone! Amazing quality!",
        "Battery life is terrible, keeps dying.",
        "Just got the new phone, still testing it out.",
        "This product is fantastic and super reliable!",
        "Really disappointed with the customer service."
    ],
    'label': ['positive', 'negative', 'neutral', 'positive', 'negative']  # Ground truth labels
}
df = pd.DataFrame(data)

# Load pre-trained BERT model and tokenizer
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Function to preprocess and predict sentiment
def predict_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Apply softmax to get probabilities
    probs = softmax(outputs.logits, dim=1)
    
    # Map model output to sentiment (1-5 star rating to positive/neutral/negative)
    score = torch.argmax(probs, dim=1).item() + 1  # Model outputs 1-5 stars
    if score >= 4:
        return "positive", probs[0][score-1].item()
    elif score == 3:
        return "neutral", probs[0][score-1].item()
    else:
        return "negative", probs[0][score-1].item()

# Apply sentiment analysis to the dataset
results = []
for text in df['text']:
    sentiment, confidence = predict_sentiment(text)
    results.append({
        'text': text,
        'predicted_sentiment': sentiment,
        'confidence': confidence
    })

# Create a DataFrame with results
results_df = pd.DataFrame(results)

# Display results
print("\nSentiment Analysis Results:")
print(results_df)

# Save results to a CSV file
results_df.to_csv('sentiment_results.csv', index=False)
print("\nResults saved to 'sentiment_results.csv'")


Sentiment Analysis Results:
                                              text predicted_sentiment  \
0  Love the camera on this phone! Amazing quality!            positive   
1           Battery life is terrible, keeps dying.            negative   
2    Just got the new phone, still testing it out.            positive   
3    This product is fantastic and super reliable!            positive   
4   Really disappointed with the customer service.            negative   

   confidence  
0    0.953644  
1    0.775541  
2    0.419054  
3    0.918771  
4    0.482650  

Results saved to 'sentiment_results.csv'
